In [1]:
# Warning: Execute this cell only once for the kernel. Reset the kernel if you need changes.
import sys
sys.path.append("../Python")
import ROOT as R
import time
import numpy as np
# Turn jsroot off if you want to make a pdf from this file.
%jsroot on
from root_helpers import SetStyle
from root_helpers import fancy_plot
from root_helpers import print_mc_particle_tree
from root_helpers import print_daughters
# R.EnableImplicitMT()

Welcome to JupyROOT 6.29/01


In [2]:
import os
recompile = True
try:
    if os.path.getmtime('../Python/Utility_Functions_C.so') - os.path.getmtime('../Python/Utility_Functions.C') > 0:
        recompile = False
        print("Recompile is not needed")
    else:
        print("Recompiling: ")
except:
    print("Recompile needed, file not found.")
if recompile:
    R.gROOT.LoadMacro("../Python/Utility_Functions.C++")
else:
    R.gSystem.Load("../Python/Utility_Functions_C.so")
R.Utility_Functions()

Recompiling: 


'Utility Functions V1.0.4 \n'

Info in <TMacOSXSystem::ACLiC>: creating shared library /System/Volumes/Data/data/HPS/Analysis/MC2021/./../Python/Utility_Functions_C.so


Info in <TMacOSXSystem::ACLiC>: creating shared library /System/Volumes/Data/data/HPS/Analysis/MC2021/./../Python/Utility_Functions_C.so


In [3]:
ch = R.TChain("MiniDST")
ch.Add("/data/HPS/data/physrun2021/MC/hpsForward_e-_3.0GeV_z0.0_*_SLIC-v06-00-01_QGSP_BERT_HPS-v2019-3pt7GeV_recon.root")
# ch.Add("/data/HPS/data/physrun2021/pass0/minidst/hps_0147*.root")
print(f"Number of events loaded: {ch.GetEntries()/1e6:7.3f}M")

Number of events loaded:   1.847M


In [4]:
df = R.RDataFrame(ch)
print("Available data names in Tuple:")
ColumnNames=df.GetColumnNames()
ll = 0
pr_colnames = [x for x in ColumnNames if str(x).startswith('')]
for nn in pr_colnames:
    if ll < len(nn):
        ll = len(nn)
for n in range(len(pr_colnames)):
    if n%4 == 0:
        print("")
    print(f"{str(pr_colnames[n]):{ll}s}",end="")

Available data names in Tuple:

ecal_cluster_energy     ecal_cluster_hits       ecal_cluster_nhits      ecal_cluster_seed_energy
ecal_cluster_seed_index ecal_cluster_seed_ix    ecal_cluster_seed_iy    ecal_cluster_time       
ecal_cluster_x          ecal_cluster_y          ecal_cluster_z          ecal_hit_energy         
ecal_hit_index_x        ecal_hit_index_y        ecal_hit_time           event_number            
ext_trigger             hodo_cluster_energy     hodo_cluster_ix         hodo_cluster_iy         
hodo_cluster_layer      hodo_cluster_time       hodo_hit_energy         hodo_hit_hole           
hodo_hit_index_x        hodo_hit_index_y        hodo_hit_layer          hodo_hit_time           
mc_part_charge          mc_part_daughters       mc_part_end_x           mc_part_end_y           
mc_part_end_z           mc_part_energy          mc_part_gen_status      mc_part_mass            
mc_part_parents         mc_part_pdg_id          mc_part_px              mc_part_py             

In [5]:
ones = None
event=108
c_name = "cc"+str(event)
cc1 = R.TCanvas("cc1","Canvas",1000,600)

In [6]:
cc1.Clear()
dfr = df.Range(event,event+1)
p0 = dfr.Display(("run_number","event_number","ecal_cluster_nhits"))
p1 = dfr.Display(("ecal_cluster_energy","ecal_cluster_time","ecal_cluster_hits","ecal_cluster_seed_index"))
p2 = dfr.Display(("ecal_hit_energy","ecal_hit_time","ecal_hit_index_x","ecal_hit_index_y"))
h_ecal = dfr.Histo2D(("h_ecal","Ecal Hits",50,-25.5,24.5,13,-6.5,6.5),"ecal_hit_index_x", "ecal_hit_index_y", "ecal_hit_energy")
# p0.Print()
# p1.Print()
# p2.Print()
h_draw = h_ecal.GetPtr()
ones = fancy_plot(h_ecal, ones, 0x0)
cc1.Draw()
# event +=1

Booking histogram h_ecal_oneslb


In [7]:
R.gSystem.Load("/data/HPS/lib/libMiniDst")

0

In [8]:
# Setup the MiniDST class, which makes for easier event by event data inplection.
# This MiniDST class looks directly into the TTree, so does not use the RDataframe class.
# You need to:
mdst = R.MiniDst()          # Initiate the class
mdst.use_mc_particles=True  # Tell it to look for the MC Particles in the TTree
mdst.DefineBranchMap()      # Define the map of all the branches to the contents of the TTree
mdst.SetBranchAddressesOnTree(ch) # Connect the TChain (which contains the TTree) to the class.

Error in <TBranchElement::InitInfo>: StreamerInfo is not compiled.


In [9]:
def make_ecal_snaphot(mini_dst, hist, opt=0):
    """Return a 2D histogram for the ECal, with the energy of the hits on the z-axis"""
    if hist is None:
        hist = R.TH2D("hist_ecal","Ecal Hits",50,-25.5,24.5,13,-6.5,6.5)
    else:
        hist.Reset()
    # Fill the histogram by looping over the ECal hits
    for i in range(len(mini_dst.ecal_hit_index_x)):
        hist.Fill(mini_dst.ecal_hit_index_x[i], mini_dst.ecal_hit_index_y[i], mini_dst.ecal_hit_energy[i])
    return hist

In [10]:
cc2 = R.TCanvas("cc2","Canvas",1000,600)
hh = None
ones2 = None
event = 108

In [11]:
ch.GetEntry(event)
hh = make_ecal_snaphot(mdst, hh, 0)
hh.SetStats(0)
cc2.Clear()
ones2 = fancy_plot(hh, ones2, 0x0)
cc2.Update()
cc2.Draw()
event += 1

Booking histogram hist_ecal_oneslb


In [12]:
print(f"Run: {mdst.run_number}, Event:{mdst.event_number}")
print(f"NClusters: {len(mdst.ecal_cluster_seed_ix)} :",end="")
for i in range(len(mdst.ecal_cluster_seed_index)):
    print(f" [{mdst.ecal_cluster_seed_index[i]}]({mdst.ecal_cluster_x[i]:7.3f},{mdst.ecal_cluster_y[i]:7.3f},{mdst.ecal_cluster_energy[i]:7.3f})", end=",")
print("\n")
print("   i |   x |   y |  Energy  ")
esum=0
print("----------------------------")
for i in range(len(mdst.ecal_hit_index_x)):
    print(f" {i:3d} | {mdst.ecal_hit_index_x[i]:3d} | {mdst.ecal_hit_index_y[i]:3d} | {mdst.ecal_hit_energy[i]:7.5f} ")
    esum+=mdst.ecal_hit_energy[i]
print(f"Energy sum = {esum}")

Run: 1193700, Event:9
NClusters: 1 : [8](154.130,-41.883,  2.834),

   i |   x |   y |  Energy  
----------------------------
   0 |   9 |  -1 | 0.07054 
   1 |  10 |  -3 | 0.03088 
   2 |   8 |  -4 | 0.03093 
   3 |   8 |  -3 | 0.17876 
   4 |   9 |  -3 | 0.11735 
   5 |   7 |  -2 | 0.06747 
   6 |   8 |  -1 | 0.05087 
   7 |   8 |  -2 | 0.84081 
   8 |   9 |  -2 | 1.14822 
Energy sum = 2.535833742469549


In [34]:
event+=1
ch.GetEntry(event)
print_mc_particle_tree(mdst)
primary_index = -1
for i in range(len(mdst.mc_part_z)):
    if abs(mdst.mc_part_z[i])<1e-6:
        primary_index = i
        break
print(f"i_p = {primary_index}")
print(f"Status: {mdst.mc_part_gen_status[primary_index]:d} = {mdst.mc_part_gen_status[primary_index]:032b}")

  12  pdg:   11  E:  3.000000 p = (-0.020064, 0.085848, 2.998704)v = ( 0.00, 0.00, 0.00) end=(-83.31,39.98,1387.73)
              | 
              5  pdg:   22  E:  0.000000 p = (-0.000000, 0.000000, 0.000000)v = (-83.31,39.98,1387.73) end=(-89.89,41.79,1461.49)
                         | 
                        13  pdg:   11  E:  0.000511 p = ( 0.000000,-0.000000, 0.000000)v = (-89.89,41.79,1461.49) end=(-90.77,41.99,1471.30)
                                    | 
                                    0  pdg:   22  E:  0.000000 p = (-0.000000, 0.000000, 0.000000)v = (-90.77,41.99,1471.30) end=(-91.17,42.11,1475.85)
                                               | 
                                              16  pdg:   11  E:  0.000511 p = (-0.000000, 0.000000, 0.000000)v = (-91.17,42.11,1475.85) end=(-91.29,42.14,1477.05)
                                                          | 
                                                          1  pdg:   22  E:  0.000000 p = (-0.000000, 0.

In [35]:
ans = R.find_average_end_of_daughters_of_primary_mc_part(mdst.mc_part_z, mdst.mc_part_end_z, mdst.mc_part_daughters)

In [36]:
list(ans)

[1461.491142325174]

In [43]:
h_n_trk = df.Define("n_trk","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n").Histo1D(("h_n_trk","N KF tracks",11,-0.5,10.5),"n_trk")

In [44]:
cc3 = R.TCanvas("cc3","Canvas",1000,600)
cc3.SetLogy()
h_n_trk.Draw()
# h_trk_type.Draw()
cc3.Draw()

In [44]:
# ch.GetEvent(1000)
print(mdst.track_type)
print(mdst.track_x_at_ecal)
print(mdst.track_y_at_ecal)
print(mdst.ecal_cluster_x)
print(mdst.ecal_cluster_y)
print(mdst.ecal_cluster_z)
print(mdst.ecal_cluster_energy)

{ 1, 32, 0 }
{ -158.68057, -159.77621, -99.000000 }
{ -32.285248, -32.322289, -99.000000 }
{ -160.90236 }
{ -31.234417 }
{ 1449.7230 }
{ 2.8540785 }


In [32]:
list(R.get_vec_abs(mdst.part.px,mdst.part.py,mdst.part.pz))

[3.0910007126754744, 3.0995140064581204]

In [43]:
print(mdst.mc_part_end_x)
print(mdst.mc_part_end_y)
print(mdst.mc_part_end_z)
print(mdst.mc_part_energy)
print_mc_particle_tree(mdst)

{ 186.77338, 172.72255, 188.34173, 127.66628, 45.715293, 190.89881, 190.28572, 188.32397, 188.81537, 190.52120, 188.07597, 172.38583, 135.35861, 190.84963 }
{ -40.967363, -115.66351, -33.819923, -88.960342, 60.178744, -35.439140, -35.194414, -34.200828, -34.532515, -35.266486, -34.296194, -115.49584, -89.120053, -35.874238 }
{ 1496.7047, 1049.6598, 1463.9988, 905.66624, 553.15587, 1483.5067, 1474.8610, 1454.3559, 1458.6116, 1478.0433, 1450.2807, 1049.6703, 901.43335, 1467.3407 }
{ 0.00051099889, 0.0023482520, 0.0000000, 0.0021482026, 0.0019986279, 0.0000000, 0.0000000, 0.00051099889, 3.0000000, 0.00051099889, 0.0000000, 0.0016679163, 0.0036247191, 0.00051099889 }
   8  pdg:   11  E:  3.000000 p = ( 0.544909,-0.072305, 2.949211)v = ( 0.00, 0.00, 0.00) end=(188.82,-34.53,1458.61)
              | 
              6  pdg:   22  E:  0.000000 p = ( 0.000000,-0.000000, 0.000000)v = (188.82,-34.53,1458.61) end=(190.29,-35.19,1474.86)
                         | 
                         9  pdg:  

In [77]:
# double_track = []
# for event in range(1,1000000):
#     ch.GetEvent(event)
#     n_kf = 0
#     for i in range(len(mdst.track_type)):
#         if mdst.track_type[i]==1:
#             n_kf+=1
#     if n_kf >= 2:
#         double_track.append(event)
# print(double_track)


[7740, 9320, 10692, 11120, 12498, 14538, 15594, 15839, 25851, 25925, 27330, 34123, 35813, 36461, 37698, 38867, 38955, 39227, 39573, 42238, 43679, 44597, 49261, 51939, 52780, 56749, 62458, 67598, 68002, 68343, 69068, 71029, 72969, 74249, 76903, 82119, 83529, 86180, 92640, 95807, 96360, 98727, 101291, 105150, 108264, 108873, 109331, 110081, 110246, 112190, 116887, 117574, 121684, 123093, 128644, 130612, 132335, 134975, 135709, 136791, 141619, 150739, 150760, 152663, 155651, 158116, 166533, 168500, 171987, 174583, 178023, 178221, 182008, 182026, 185742, 185959, 185968, 187362, 188848, 190582, 195068, 197136, 197217, 197492, 198047, 198570, 199903, 200578, 200763, 201570, 206050, 206181, 207824, 208582, 210321, 211882, 216215, 216751, 222436, 227722, 228194, 229487, 230785, 231485, 233605, 233656, 234932, 235671, 244457, 247981, 248278, 251273, 254685, 258532, 259068, 260481, 263988, 265466, 265866, 266630, 268555, 270973, 271275, 273286, 273612, 274642, 276638, 277130, 277660, 288698, 290

In [31]:
dfx = df.Define("n_kf_track","int n=0; for(int i=0;i<track_type.size();++i){if(track_type[i]==1) n+=1;}; return n")\
    .Filter("n_kf_track==1").Filter("ecal_cluster_energy.size()==1")\
    .Define("trk_idx","for(int i=0;i<track_type.size();++i){if(track_type[i]==1) return(i);} return(-1);") \
    .Define("diff_posx_ecal","ecal_cluster_x[0] - track_x_at_ecal[trk_idx]") \
    .Define("diff_posy_ecal","ecal_cluster_y[0] - track_y_at_ecal[trk_idx]") \

cnt  = df.Count()
cntx = dfx.Count()

In [38]:
hx_diff_posx_ecal=dfx.Histo1D(("hx_diff_posx","Diff Ecal_x - Track_x",1000,-20.,20.),"diff_posx_ecal")

In [32]:
print(cnt.GetValue())
print(cntx.GetValue())

1846511
1104691


In [40]:
cc4 = R.TCanvas("cc4","Canvas",1000,600)
cc4.Divide(2,2)
p1=cc4.cd(1)
#p1.SetLogy()
hx_diff_posx_ecal.Draw()
p2=cc4.cd(2)
p2.SetLogy()
p3=cc4.cd(3)
p3.SetLogy()
p4=cc4.cd(4)
p4.SetLogy()
# h_trk_type.Draw()
cc4.Draw()

Warning in <TCanvas::Constructor>: Deleting canvas with same name: cc4
